In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        do_value=0.02
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(3,3), padding=0, stride=1), 
            nn.BatchNorm2d(20),
            nn.Dropout(do_value),
            nn.ReLU()
        )
        #Output=26 RF=3X3 [RFin + (Ksize-1 * JMPin) => 1+(3-1)*1 =3]  :JMPin=1, Jout= JMPin X s = 1
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=16, kernel_size=(3,3), padding=0, stride=1),
            nn.BatchNorm2d(16),
            nn.Dropout(do_value),
            nn.ReLU()    
        )
        #Output=24 RF=5X5  [RFin + (Ksize-1 * JMPin) => 3+(3-1)*1 =5] :JMPin=1, Jout =JMPin X s =1

        self.pool1 = nn.MaxPool2d(2, 2)#Output=12 RF=6X6 [RFin + (Ksize-1 * JMPin) => 5+(2-1)*1 =6] :JMPin=1, Jout=  JMPin X s =2

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), padding=0, stride=1),
            nn.BatchNorm2d(16),
            nn.Dropout(do_value),
            nn.ReLU()
        )
        #Output=10 RF=10X10 [RFin + (Ksize-1 * JMPin) => 6+(3-1)*2 =10] : Jout= JMPin X s = 2X1 :JMPin=2, Jout= JMPin X s = 2X1=2

        self.pool2 = nn.MaxPool2d(2, 2) #Output=5 RF=12[RFin + (Ksize-1 * JMPin) => 10+(2-1)*2 =12]  :JMPin=2, Jout =JMPin X s = 2X2 =4

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(3,3), padding=0, stride=1),
            nn.BatchNorm2d(16),
            nn.Dropout(do_value),
            nn.ReLU()            
        )
        #Output=3 RF= 20[RFin + (Ksize-1 * JMPin) => 12+(3-1)*4 =20] :JMPin=4, Jout =JMPin X s = 4X1=4
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=13, kernel_size=(3,3), padding=0, stride=1),
            nn.BatchNorm2d(13),
            nn.Dropout(do_value),
            nn.ReLU()
            
        )
        #Output=1 RF=28 [RFin + (Ksize-1 * JMPin) => 20+(3-1)*4 =28]  :JMPin=4, Jout=JMPin X s = 4X1=4
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=13, out_channels=10, kernel_size=(1,1), padding=0, stride=1)
            
        )
        
     

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.conv5(x)   
        x = self.conv6(x)      
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [66]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 26, 26]             200
       BatchNorm2d-2           [-1, 20, 26, 26]              40
           Dropout-3           [-1, 20, 26, 26]               0
              ReLU-4           [-1, 20, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,896
       BatchNorm2d-6           [-1, 16, 24, 24]              32
           Dropout-7           [-1, 16, 24, 24]               0
              ReLU-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 16, 10, 10]           2,320
      BatchNorm2d-11           [-1, 16, 10, 10]              32
          Dropout-12           [-1, 16, 10, 10]               0
             ReLU-13           [-1, 16, 10, 10]               0
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm

Train_Loss=[]
Train_Accuracy=[]
Test_Loss=[]
Test_Accuracy=[]

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct_train=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        Train_Loss.append(loss.item())
        pred = output.argmax(dim=1, keepdim=True)
        correct_train += pred.eq(target.view_as(pred)).sum().item()
        Train_Accuracy.append(100.00 *correct_train/len(train_loader.dataset))
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'TRAINING Loss={loss.item()} batch_id={batch_idx}')
        
        ##Added detailed percentage:
        pbar.set_description(desc= f'TRAIN Loss={loss.item()} batch_id={batch_idx} Correct={correct_train} / {len(train_loader.dataset)} TRAIN ACCURACY={100.00 *correct_train/len(train_loader.dataset)}')
     


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss=F.nll_loss(output, target)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    Test_Loss.append(test_loss)
    Test_Accuracy.append(100. * correct / len(test_loader.dataset))
    print('\nTEST: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    


In [69]:
import pandas as pd
import numpy as np

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
df_Test_Train_Analysis=pd.DataFrame(columns=['EPOCH','Test_Acc','Train_Acc','Acc_Diff'])
print(df_Test_Train_Analysis.shape)
for epoch in range(1, 15):
    print('EPOCH #',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print('----------------------------------------------------------------------')

  0%|          | 0/938 [00:00<?, ?it/s]

(0, 4)
EPOCH # 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
TRAIN Loss=0.016629531979560852 batch_id=937 Correct=56087 / 60000 TRAIN ACCURACY=93.47833333333334: 100%|██████████| 938/938 [00:24<00:00, 38.02it/s]
  0%|          | 0/938 [00:00<?, ?it/s]


TEST: Average loss: 0.0531, Accuracy: 9840/10000 (98.40%)

----------------------------------------------------------------------
EPOCH # 2


TRAIN Loss=0.04653531312942505 batch_id=937 Correct=58812 / 60000 TRAIN ACCURACY=98.02: 100%|██████████| 938/938 [00:24<00:00, 38.22it/s]
  0%|          | 0/938 [00:00<?, ?it/s]

...


TEST: Average loss: 0.0267, Accuracy: 9914/10000 (99.14%)

----------------------------------------------------------------------
EPOCH # 13


TRAIN Loss=0.005518272519111633 batch_id=937 Correct=59578 / 60000 TRAIN ACCURACY=99.29666666666667: 100%|██████████| 938/938 [00:24<00:00, 38.17it/s]
  0%|          | 0/938 [00:00<?, ?it/s]


TEST: Average loss: 0.0265, Accuracy: 9921/10000 (99.21%)

----------------------------------------------------------------------
EPOCH # 14


TRAIN Loss=0.007515609264373779 batch_id=937 Correct=59560 / 60000 TRAIN ACCURACY=99.26666666666667: 100%|██████████| 938/938 [00:24<00:00, 38.56it/s]



TEST: Average loss: 0.0277, Accuracy: 9917/10000 (99.17%)

----------------------------------------------------------------------


#Goal	
In this iteration we want to -
1 - increase the Model capacity.
2 - Try reducing Batch size to 64
#Params	
9,923

#WITH 15 EPOCHS

#DropOut value
2 %
#Best Train Accuracy	
99.29%

#Best Test Accuracy	
99.32%
#Observation/ Analysis/Conclusion	
The model is not Overfitting, but we are yet to attain Target Accuracy. 

#Comment	
In next iteration, we shall try to manipulate the Learning Rate